In [36]:
import pandas as pd
import numpy as np
import os

In [ ]:
current_dir = os.getcwd()
url_brent_oil = os.path.join(current_dir, "../data/raw/macrodata/brent_oil.csv")
url_consumo_nacional_bruto = os.path.join(current_dir, "../data/raw/macrodata/Consumo nacional bruto.csv")
url_cpi = os.path.join(current_dir, "../data/raw/macrodata/cpi(%).csv")
url_ingreso_nacional_bruto = os.path.join(current_dir, "../data/raw/macrodata/ingreso_nacional_bruto.csv")
url_sueldo_minimo_interprofesional = os.path.join(current_dir, "../data/raw/macrodata/sueldo_minimo_interprofesional_2018.csv")
url_tasa_interes = os.path.join(current_dir, "../data/raw/macrodata/tasa_interes.csv")
url_categories_unemployed_total_index = os.path.join(current_dir, "../data/raw/macrodata/unemployed_total_index(%).csv")


## Brent Oil

In [38]:
brent_oil_raw = pd.read_csv(url_brent_oil, sep = ",")
brent_oil_raw

,Date,Open,High,Low,Close,Adj Close,Volume
0,"May 8, 2018",75.66,76.18,73.10,74.85,74.85,"50,813"
1,"May 7, 2018",74.98,76.33,74.85,76.17,76.17,"21,266"
2,"May 4, 2018",73.76,75.05,73.25,74.87,74.87,"30,457"
3,"May 3, 2018",73.14,74.08,72.48,73.62,73.62,"35,839"
4,"May 2, 2018",73.21,73.62,72.37,73.36,73.36,"34,769"
...,...,...,...,...,...,...,...
83,"Jan 8, 2018",67.82,67.98,67.45,67.78,67.78,"17,704"
84,"Jan 5, 2018",68.06,68.09,67.27,67.62,67.62,"18,290"
85,"Jan 4, 2018",68.00,68.26,67.53,68.07,68.07,"22,840"
86,"Jan 3, 2018",66.49,68.01,66.40,67.84,67.84,"25,878"


In [39]:
brent_oil = brent_oil_raw[["Date", "Close"]]
brent_oil

,Date,Close
0,"May 8, 2018",74.85
1,"May 7, 2018",76.17
2,"May 4, 2018",74.87
3,"May 3, 2018",73.62
4,"May 2, 2018",73.36
...,...,...
83,"Jan 8, 2018",67.78
84,"Jan 5, 2018",67.62
85,"Jan 4, 2018",68.07
86,"Jan 3, 2018",67.84


##  Average Propensity to Consume (APC / PMC) = Consumo Nacional Bruto/Ingreso Nacional Bruto

Generamos la columna APC (Average Propensity to Consume) realizando la división del Consumo Nacional Bruto entre Ingreso Nacional Bruto. Ambos extraídos del CSV.

In [ ]:
consumo_nacional_bruto = pd.read_csv(url_consumo_nacional_bruto, sep = ",")
ingreso_nacional_bruto = pd.read_csv(url_ingreso_nacional_bruto, sep = ",")

# Realizamos un merge para unir los dos dataframes a través de Country (el parámetro how="inner" 
# indica como quieres que se una, por los que tienen el mismo Country en las dos columnas)
apc = pd.merge(consumo_nacional_bruto, ingreso_nacional_bruto, on="Country", how = "inner")

# Se quita los que eran ".."
apc = apc[(apc['Consumo Total Hogares 2018'] != '..') & (apc['Ingreso nacional bruto 2018'] != '..')]

# Convertir los datos de Consumo e Ingreso de String a Float
apc[["Consumo Total Hogares 2018", "Ingreso nacional bruto 2018"]] = apc[["Consumo Total Hogares 2018", "Ingreso nacional bruto 2018"]].astype(float)

# Realizamos la división
apc["APC_%"] = apc["Consumo Total Hogares 2018"]/apc["Ingreso nacional bruto 2018"]
apc = apc[["Country", "APC_%"]]
apc

,Country,APC_%
1,Albania,0.745635
2,Algeria,0.426420
5,Angola,0.610799
7,Argentina,0.720406
8,Armenia,0.785379
...,...,...
261,Sub-Saharan Africa,0.755461
262,Sub-Saharan Africa (excluding high income),0.755464
263,Sub-Saharan Africa (IDA & IBRD countries),0.755461
264,Upper middle income,0.481546


## Realizamos el merge a pares de datos 
Siguiendo la misma función de merge, ir uniendo cada par de datos con el siguiente dato.

## CPI + APC

In [ ]:
url_cpi = os.path.join(current_dir, "../data/raw/macrodata/cpi(%).csv")
cpi = pd.read_csv(url_cpi, sep = ",")
cpi

,Country,cpi_2018_%
0,Aruba,3.626041
1,Africa Eastern and Southern,4.720805
2,Afghanistan,0.626149
3,Africa Western and Central,1.784050
4,Angola,19.628938
...,...,...
261,Kosovo,1.053798
262,"Yemen, Rep.",NaN
263,South Africa,4.509873
264,Zambia,7.494572


In [51]:
merge_cpi_apc = pd.merge(apc, cpi, on="Country", how = "outer")
merge_cpi_apc

,Country,APC_%,cpi_2018_%
0,Afghanistan,NaN,0.626149
1,Africa Eastern and Southern,0.693258,4.720805
2,Africa Western and Central,NaN,1.784050
3,Albania,0.745635,2.028060
4,Algeria,0.426420,4.269990
...,...,...,...
261,West Bank and Gaza,0.758970,-0.195108
262,World,0.562512,2.442583
263,"Yemen, Rep.",1.300891,NaN
264,Zambia,0.456030,7.494572


## CPI + APC + Tasa Interés

In [47]:
url_tasa_interes = os.path.join(current_dir, "../data/raw/macrodata/tasa_interes.csv")
tasa_interes = pd.read_csv(url_tasa_interes, sep = ",")
merge_cpi_apc_ti = pd.merge(merge_cpi_apc, tasa_interes, on="Country", how = "outer")
merge_cpi_apc_ti

,Country,APC_%,cpi_2018_%,Tasas_de_interes_%
0,Afghanistan,NaN,0.626149,NaN
1,Africa Eastern and Southern,0.693258,4.720805,NaN
2,Africa Western and Central,NaN,1.784050,NaN
3,Albania,0.745635,2.028060,2.880525
4,Algeria,0.426420,4.269990,1.728750
...,...,...,...,...
261,West Bank and Gaza,0.758970,-0.195108,6.193658
262,World,0.562512,2.442583,NaN
263,"Yemen, Rep.",1.300891,NaN,NaN
264,Zambia,0.456030,7.494572,2.215865


## CPI + APC + Tasa Interés + Indice Total de Desempleo

In [52]:
url_categories_unemployed_total_index = os.path.join(current_dir, "../data/raw/macrodata/unemployed_total_index(%).csv")
unemployed_total_index = pd.read_csv(url_categories_unemployed_total_index, sep = ",")
merge_cpi_apc_ti_itd = pd.merge(merge_cpi_apc, tasa_interes, on="Country", how = "outer")
merge_cpi_apc_ti_itd = pd.merge(merge_cpi_apc_ti, unemployed_total_index, on = "Country", how="outer")
merge_cpi_apc_ti_itd

,Country,APC_%,cpi_2018_%,Tasas_de_interes_%,unemployed_index_2018_%
0,Afghanistan,NaN,0.626149,NaN,11.057000
1,Africa Eastern and Southern,0.693258,4.720805,NaN,NaN
2,Africa Western and Central,NaN,1.784050,NaN,NaN
3,Albania,0.745635,2.028060,2.880525,12.340000
4,Algeria,0.426420,4.269990,1.728750,11.882000
...,...,...,...,...,...
270,West Bank and Gaza,0.758970,-0.195108,6.193658,26.256001
271,World,0.562512,2.442583,NaN,5.391889
272,"Yemen, Rep.",1.300891,NaN,NaN,13.002000
273,Zambia,0.456030,7.494572,2.215865,11.500000


## CPI + APC + Tasa Interés + Indice Total de Desempleo + Sueldo Mínimo Interprofesional [PENDIENTE]

In [ ]:
# url_sueldo_minimo_interprofesional = os.path.join(current_dir, "../data/raw/macrodata/sueldo_minimo_interprofesional_2018.csv")
# url_sueldo_minimo_interprofesional = pd.read_csv(url_sueldo_minimo_interprofesional, sep = ",")
# url_sueldo_minimo_interprofesional.rename(columns = { "Countries" : "Country" }, inplace = True)
# merge_cpi_apc_smi = pd.merge(merge_cpi_apc, url_sueldo_minimo_interprofesional, on = "Country", how="outer")
# merge_cpi_apc_smi

,Country,APC_%,cpi_2018_%,SMI (euros)
0,Afghanistan,NaN,0.626149,0.0
1,Africa Eastern and Southern,0.693258,4.720805,NaN
2,Africa Western and Central,NaN,1.784050,NaN
3,Albania,0.745635,2.028060,0.0
4,Algeria,0.426420,4.269990,0.0
...,...,...,...,...
285,West Bank and Gaza,0.758970,-0.195108,NaN
286,World,0.562512,2.442583,NaN
287,"Yemen, Rep.",1.300891,NaN,NaN
288,Zambia,0.456030,7.494572,0.0
